In [3]:
import psycopg2

# Connect to the default "postgres" database to create a new one
conn = psycopg2.connect(
    host="34.47.32.127",
    port=5432,  # Default PostgreSQL port
    dbname="octogone_cat_v0",
    user="postgres",
    password="f87JpR9Uvud6NR3HwbP"
)


# INSERT FAMILIES

In [7]:
import pandas as pd

classif_df = pd.read_csv('../Classification_reference.csv')
print(classif_df.head())

                   FamilyTitle  FamilyCode  \
0  Food/Beverage Variety Packs    50230000   
1                    Beverages    50200000   
2                    Beverages    50200000   
3                    Beverages    50200000   
4                    Beverages    50200000   

                                          ClassTitle  ClassCode  \
0                        Food/Beverage Variety Packs   50230100   
1  Alcoholic Beverages (Includes De-Alcoholised V...   50202200   
2                             Coffee/Tea/Substitutes   50201700   
3       Non Alcoholic Beverages - Not Ready to Drink   50202400   
4           Non Alcoholic Beverages - Ready to Drink   50202300   

                                          BrickTitle  BrickCode  \
0                        Food/Beverage Variety Packs   10000590   
1                  Alcoholic Beverages Variety Packs   10000591   
2               Coffee/Tea/Substitutes Variety Packs   10000592   
3  Non Alcoholic Beverages Variety Packs - Not Re...

In [9]:
# Select unique FamilyTitle And FamilyCode from the classif_df data frame and add a uuid4 column
import uuid
classif_df = classif_df[['FamilyTitle', 'FamilyCode']].drop_duplicates()
classif_df['uuid'] = [uuid.uuid4() for _ in range(len(classif_df.index))]
print(classif_df.head())

                                FamilyTitle  FamilyCode  \
0               Food/Beverage Variety Packs    50230000   
1                                 Beverages    50200000   
5                     Bread/Bakery Products    50180000   
9               Cereal/Grain/Pulse Products    50220000   
12  Confectionery/Sugar Sweetening Products    50160000   

                                    uuid  
0   08ba177a-19c8-4856-8891-72a318aef1f3  
1   c44011b6-2458-4147-aa80-637bb210fbed  
5   3e25f14a-5b90-4d65-b84c-bd64c26c7b24  
9   f142c2b0-bcd4-4a57-bfbf-0fc61f4406a9  
12  72256ba3-87db-484d-87b3-d17bcd9454c5  


In [16]:
# Insert the data into the catalog.family table
conn.rollback()
cursor = conn.cursor()
for i, row in classif_df.iterrows():
    cursor.execute(
        "INSERT INTO catalog.family (family_id, family_name_en, family_code, family_kind) VALUES (%s, %s, %s, %s)",
        (str(row['uuid']), row['FamilyTitle'], row['FamilyCode'], 'FOOD')
    )
conn.commit()
cursor.close()

# For each family insert the categories

In [17]:
import pandas as pd

classif_df = pd.read_csv('../Classification_reference.csv')
print(classif_df.head())

                   FamilyTitle  FamilyCode  \
0  Food/Beverage Variety Packs    50230000   
1                    Beverages    50200000   
2                    Beverages    50200000   
3                    Beverages    50200000   
4                    Beverages    50200000   

                                          ClassTitle  ClassCode  \
0                        Food/Beverage Variety Packs   50230100   
1  Alcoholic Beverages (Includes De-Alcoholised V...   50202200   
2                             Coffee/Tea/Substitutes   50201700   
3       Non Alcoholic Beverages - Not Ready to Drink   50202400   
4           Non Alcoholic Beverages - Ready to Drink   50202300   

                                          BrickTitle  BrickCode  \
0                        Food/Beverage Variety Packs   10000590   
1                  Alcoholic Beverages Variety Packs   10000591   
2               Coffee/Tea/Substitutes Variety Packs   10000592   
3  Non Alcoholic Beverages Variety Packs - Not Re...

In [18]:
# Insert the data into the catalog.category table
# conn.rollback()
cursor = conn.cursor()

# Retrieve family_id from catalog.family using family_code
family_code_to_id = {}
cursor.execute("SELECT family_code, family_id FROM catalog.family")
for record in cursor.fetchall():
    family_code_to_id[record[0]] = record[1]

# Insert categories
for i, row in classif_df.iterrows():
    family_id = family_code_to_id.get(row['FamilyCode'])
    if family_id:
        cursor.execute(
            "INSERT INTO catalog.category (category_id, family_id, category_name_en, category_code) VALUES (%s, %s, %s, %s)",
            (str(uuid.uuid4()), family_id, row['ClassTitle'], row['ClassCode'])
        )

conn.commit()
cursor.close()

# Insert subcategories

Subcategories do not align with a single column.  Each family must be explored.

In [20]:
# Insert Subcategories

import pandas as pd

classif_df = pd.read_csv('../Classification_reference.csv')

unique_families_df = classif_df[['FamilyTitle', 'FamilyCode']].drop_duplicates()
print(unique_families_df.sort_values(by='FamilyCode'))



                                           FamilyTitle  FamilyCode
14     Fruits/Vegetables/Nuts/Seeds Prepared/Processed    50100000
23                                    Fish and Seafood    50120000
16   Milk/Butter/Cream/Yogurts/Cheese/Eggs/Substitutes    50130000
17                                    Oils/Fats Edible    50150000
12             Confectionery/Sugar Sweetening Products    50160000
24                   Seasonings/Preservatives/Extracts    50170000
5                                Bread/Bakery Products    50180000
19                            Prepared/Preserved Foods    50190000
1                                            Beverages    50200000
9                          Cereal/Grain/Pulse Products    50220000
0                          Food/Beverage Variety Packs    50230000
33             Fruits - Unprepared/Unprocessed (Fresh)    50250000
32   Vegetables (Non Leaf) - Unprepared/Unprocessed...    50260000
43    Leaf Vegetables - Unprepared/Unprocessed (Fresh)    5035

In [21]:
sub_family_df = classif_df[classif_df['FamilyCode'] == 50100000]

query = """
    INSERT INTO catalog.subcategory 
        (sub_category_id, 
        sub_category_name_en, 
        sub_category_code, 
        category_id, 
        searchable_text)
    """



                                        FamilyTitle  FamilyCode  \
14  Fruits/Vegetables/Nuts/Seeds Prepared/Processed    50100000   
15  Fruits/Vegetables/Nuts/Seeds Prepared/Processed    50100000   

                                    ClassTitle  ClassCode  \
14                Fruit/Nuts/Seeds Combination   50101900   
15  Fruits/Vegetables/Nuts/Seeds Variety Packs   50102200   

                                    BrickTitle  BrickCode AttributeTitle  \
14  Fruit/Nuts/Seeds Combination Variety Packs   10000604            NaN   
15  Fruits/Vegetables/Nuts/Seeds Variety Packs   10000605            NaN   

    AttributeCode AttributeValueTitle  AttributeValueCode  \
14            NaN                 NaN                 NaN   
15            NaN                 NaN                 NaN   

   AttributeValueDefinition                             searchable_text  \
14                      NaN  FRUIT NUTS SEEDS COMBINATION VARIETY PACKS   
15                      NaN  FRUITS VEGETABLES NUTS